<a href="https://colab.research.google.com/github/respwill/ColoradoBoulder/blob/main/Unsupervised_week4_STEP2_3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Import dependencies


In [ ]:
import pandas as pd
import numpy as np
import itertools
from tqdm import tqdm
tqdm.pandas()
from sklearn.metrics import accuracy_score, confusion_matrix, f1_score
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.decomposition import NMF
from sklearn.model_selection import RandomizedSearchCV
from collections import Counter
import re
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.tokenize import sent_tokenize, word_tokenize
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('wordnet')
nltk.download('omw-1.4')

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!
[nltk_data] Downloading package omw-1.4 to /root/nltk_data...
[nltk_data]   Package omw-1.4 is already up-to-date!


True

# Read preprocessed dataset

In [ ]:
train = pd.read_csv('/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Unsupervised learning/week4_peer_review/preprocessed_train.csv')

In [ ]:
train.head()

,Unnamed: 0,ArticleId,Text,Category,n_word_token,word_tokens,majority,majority_n,no_punct,no_punct_num,no_stopwords,clean_text,tokenized,lemmatized,lem_sent,n_words
0,0,1833,worldcom ex-boss launches defence lawyers defe...,business,18,"['last', 'week', 'mci', 'agreed', 'to', 'a', '...",a,2,worldcom exboss launches defence lawyers defen...,worldcom exboss launches defence lawyers defen...,worldcom exboss launches defence lawyers defen...,worldcom exboss launches defence lawyers defen...,"['worldcom', 'exboss', 'launches', 'defence', ...","['worldcom', 'exboss', 'launch', 'defence', 'l...",worldcom exboss launch defence lawyer defendin...,185
1,1,154,german business confidence slides german busin...,business,28,"['eurozone', 'interest', 'rates', 'are', 'at',...",interest,2,german business confidence slides german busin...,german business confidence slides german busin...,german business confidence slides german busin...,german business confidence slides german busin...,"['german', 'business', 'confidence', 'slides',...","['german', 'business', 'confidence', 'slide', ...",german business confidence slide german busine...,194
2,2,1101,bbc poll indicates economic gloom citizens in ...,business,14,"['in', 'eight', 'of', 'the', 'countries', 'the...",the,2,bbc poll indicates economic gloom citizens in ...,bbc poll indicates economic gloom citizens in ...,bbc poll indicates economic gloom citizens maj...,bbc poll indicates economic gloom citizens maj...,"['bbc', 'poll', 'indicates', 'economic', 'gloo...","['bbc', 'poll', 'indicates', 'economic', 'gloo...",bbc poll indicates economic gloom citizen majo...,267
3,3,1976,lifestyle governs mobile choice faster bett...,tech,23,"['by', 'contrast', 'he', 'said', 'camera', 'ph...",were,2,lifestyle governs mobile choice faster bett...,lifestyle governs mobile choice faster bett...,lifestyle governs mobile choice faster better ...,lifestyle governs mobile choice faster better ...,"['lifestyle', 'governs', 'mobile', 'choice', '...","['lifestyle', 'governs', 'mobile', 'choice', '...",lifestyle governs mobile choice faster better ...,344
4,4,917,enron bosses in $168m payout eighteen former e...,business,42,"['the', 'university', 'of', 'california', 'sai...",the,5,enron bosses in 168m payout eighteen former en...,enron bosses in m payout eighteen former enron...,enron bosses payout eighteen former enron dire...,enron bosses payout eighteen former enron dire...,"['enron', 'bosses', 'payout', 'eighteen', 'for...","['enron', 'boss', 'payout', 'eighteen', 'forme...",enron boss payout eighteen former enron direct...,198


In [ ]:
train.shape

(1484, 16)

In [ ]:
train_df = train.copy()

# Non negative Matrix Factorization & Prediction & Permutation

In [ ]:
# Matrix has 5 scores for each observations, and index of the biggest value is prediciton
def predict(matrix):
    return np.argmax(matrix, axis=1)

In [ ]:
# Eventhough model assign some values, but we have no idea which Category should match to each index
# As iterating the permutation of index, we check what combination gives the best accuracy
def label_permute(df, pred_array, permute_list=[], n=5):
    target_labels = train.Category.unique()
    # Permutation list is not given, it needs to check combination
    if permute_list == []:
        permute_list = list(itertools.permutations(range(n)))
        pbar = tqdm(permute_list)
        best_acc = 0
        best_label = []
        for perm in pbar:
            label_candidate = {label:i for label,i in zip(target_labels, perm)}
            df['target'] = df['Category'].map(label_candidate)
            acc = accuracy_score(df['target'], pred_array)
            if acc >= best_acc:
                best_acc = acc
                best_label = perm
                df['best'] = df['target']
        return best_label, best_acc, df
    # Previously checked permutation is given, assign label by the permutation
    else:
        if 'Category' in df.columns:
            # edit the category as numerial label
            label_candidate = {label:i for label,i in zip(target_labels, permute_list)}
            df['target'] = df['Category'].map(label_candidate)
            # calculate accuracy comparing target and prediction
            acc = accuracy_score(df['target'], pred_array)
            return permute_list, acc, df
        else:
            # In case of test dataset that has no Category column (target)
            df['pred'] =  pred_array
            # convert prediction into string category to submit to Kaggle
            label_candidate = {i:label for label,i in zip(target_labels, permute_list)}
            df['pred'] = df['pred'].apply(lambda x: label_candidate[x])
            return permute_list, 0, df

    return best_label, best_acc, df

In [ ]:
# create dictionary which give index of each category by permutation list
def get_permute_dict(df, perm):
    target_labels = df.Category.unique()
    perm_dict = {label:i for label,i in zip(target_labels, perm)}
    return perm_dict

In [ ]:
# predict by Non negative Matrix Factorization
def predict_nmf(df, param, tfidvec='', nmf_model=''):
    max_v = param['max_df']
    min_v = param['min_df']
    norm = param['norm']
    permute_list = param['permutation']
    # Train new tfidfvectorizer if a vectorizer is not given
    if tfidvec == '':
        tfidvec = TfidfVectorizer(stop_words = 'english', max_df=max_v, min_df=min_v, norm=norm)
        tfidvec_df = tfidvec.fit_transform(df['lem_sent'])
    else:
        tfidvec_df = tfidvec.transform(df['lem_sent'])

    # If NMF model is not given, train new one.
    if nmf_model == '':
        nmf_model = NMF(n_components=5,
                        random_state=4200
                        )
        nmf_model.fit(tfidvec_df)
    else:
        pass

    vector = nmf_model.transform(tfidvec_df)

    # predict by vector using argmax
    pred = predict(vector)

    # permute label of the prediction
    if permute_list != []:
        label, acc, df = label_permute(df, pred, permute_list)
    else:
        label, acc, df = label_permute(df, pred)
    return label, acc, pred, df, tfidvec, nmf_model

## Search around combination of hyperparameters of TfidfVectorizer

In [ ]:
# find best parameter combination through for loop
best_acc = 0
best_parameters = {}
best_tfidvec = ''
best_nmf = ''
for norm in ['l1','l2']:
    for max_v in np.arange(0,1,0.05):
        for min_v in np.arange(0,5):
            try:
                param = {'max_df':max_v, 'min_df':min_v, 'norm':norm, 'permutation':[]}
                label, acc, pred, train_df, tfidvec, nmf = predict_nmf(train_df, param)
                if acc >= best_acc:
                    best_acc = acc
                    best_parameters['permutation'] = label
                    best_parameters['norm'] = norm
                    best_parameters['max_df'] = max_v
                    best_parameters['min_df'] = min_v
                    best_tfidvec = tfidvec
                    best_nmf = nmf
                    print("best so far")
                    print(norm, max_v, min_v, label)
                    print(acc)
            except:
                continue
print(best_parameters)
print(best_acc)

/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
100%|██████████| 120/120 [00:00<00:00, 717.21it/s]


best so far
l1 0.05 0 (0, 2, 1, 4, 3)
0.6482479784366577


/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
100%|██████████| 120/120 [00:00<00:00, 767.49it/s]


best so far
l1 0.05 1 (0, 2, 1, 4, 3)
0.6482479784366577


/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
100%|██████████| 120/120 [00:00<00:00, 690.99it/s]


...

best so far
l1 0.15000000000000002 0 (2, 4, 1, 0, 3)
0.9386792452830188


/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
100%|██████████| 120/120 [00:00<00:00, 738.08it/s]


best so far
l1 0.15000000000000002 1 (2, 4, 1, 0, 3)
0.9386792452830188


/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
100%|██████████| 120/120 [00:00<00:00, 766.11it/s]
/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
100%|██████████| 120/120 [00:00<00:00, 693.37it/s]
/usr/local/lib/python3.8/dist-packages/sklearn/decomposition/_nmf.py:289: FutureWarning: The 'init' value, when 'init=None' and n_components is less than n_samples and n_features, will be changed from 'nndsvd' to 'nndsvda' in 1.1 (renaming of 0.26).
  warnings.warn(
100%|██████████| 120/120 [00:00<00:00, 715.39it/s]
/usr/local/lib/python3.8/dist-packages/sklearn

{'permutation': (2, 4, 1, 0, 3), 'norm': 'l1', 'max_df': 0.15000000000000002, 'min_df': 1}
0.9386792452830188


# Evaluation

In [ ]:
# evaluate the results using metrics: confusion matrix, f1 score, accuracy
def evaluate_results(y_true, y_pred):
    conf_matrix = confusion_matrix(y_true, y_pred)
    f1 = f1_score(y_true, y_pred, average='weighted')
    acc = accuracy_score(y_true, y_pred)
    print("Accuracy:", acc)
    print("F1 score:", f1)
    print("Confusion Matrix:")
    print(conf_matrix)

### Performance on Train dataset

In [ ]:
label, acc, pred, train, _, _ = predict_nmf(train, best_parameters, best_tfidvec, best_nmf)
perm_dict = get_permute_dict(train, best_parameters['permutation'])
train['best'] = train['Category'].map(perm_dict)
evaluate_results(train['best'], pred)

Accuracy: 0.9386792452830188
F1 score: 0.9386010907627307
Confusion Matrix:
[[341   0   0   3   0]
 [  5 246  17   1   3]
 [  1   5 319   2   9]
 [  0   2   7 248  15]
 [  2   4   4  11 239]]


### Submit results to Kaggle

In [ ]:
test = pd.read_csv("/content/drive/MyDrive/Kaggle/learn-ai-bbc/BBC News Test.csv")

In [ ]:
def preprocessing(df, target_col):
    df['no_punct'] = df[target_col].progress_apply(lambda x: re.sub(r'[^\w\s]+', '', x))
    df['no_punct_num'] = df['no_punct'].progress_apply(lambda x: re.sub(r'[0-9]+', '', x))
    stop_words = stopwords.words('english')
    df['no_stopwords'] = df['no_punct_num'].progress_apply(lambda x: ' '.join([word for word in x.split() if word not in (stop_words)]))
    df['clean_text'] = df['no_stopwords'].progress_apply(lambda x: re.sub(' +', ' ', x))
    df['tokenized'] = df['clean_text'].progress_apply(lambda x: word_tokenize(x))
    word_lemmatizer = WordNetLemmatizer()
    df['lemmatized'] = df['tokenized'].progress_apply(lambda x: [word_lemmatizer.lemmatize(word.lower()) for word in x])
    df['lem_sent'] = df['lemmatized'].progress_apply(lambda x: ' '.join(x))
    df['n_words'] = df['lemmatized'].progress_apply(lambda x: len(x))

In [ ]:
# preprocess the test data using same way that train data does
preprocessing(test, 'Text')

100%|██████████| 735/735 [00:00<00:00, 308713.54it/s]


In [ ]:
# Predict category using the best parameters, the best tfidfvectorizer and best NMF model
test_label, test_acc, test_pred, test, _, _ = predict_nmf(test, best_parameters, best_tfidvec, best_nmf)

In [ ]:
test.head()

,ArticleId,Text,no_punct,no_punct_num,no_stopwords,clean_text,tokenized,lemmatized,lem_sent,n_words,pred
0,1018,qpr keeper day heads for preston queens park r...,qpr keeper day heads for preston queens park r...,qpr keeper day heads for preston queens park r...,qpr keeper day heads preston queens park range...,qpr keeper day heads preston queens park range...,"[qpr, keeper, day, heads, preston, queens, par...","[qpr, keeper, day, head, preston, queen, park,...",qpr keeper day head preston queen park ranger ...,86,sport
1,1319,software watching while you work software that...,software watching while you work software that...,software watching while you work software that...,software watching work software monitor every ...,software watching work software monitor every ...,"[software, watching, work, software, monitor, ...","[software, watching, work, software, monitor, ...",software watching work software monitor every ...,232,tech
2,1138,d arcy injury adds to ireland woe gordon d arc...,d arcy injury adds to ireland woe gordon d arc...,d arcy injury adds to ireland woe gordon d arc...,arcy injury adds ireland woe gordon arcy ruled...,arcy injury adds ireland woe gordon arcy ruled...,"[arcy, injury, adds, ireland, woe, gordon, arc...","[arcy, injury, add, ireland, woe, gordon, arcy...",arcy injury add ireland woe gordon arcy ruled ...,135,sport
3,459,india s reliance family feud heats up the ongo...,india s reliance family feud heats up the ongo...,india s reliance family feud heats up the ongo...,india reliance family feud heats ongoing publi...,india reliance family feud heats ongoing publi...,"[india, reliance, family, feud, heats, ongoing...","[india, reliance, family, feud, heat, ongoing,...",india reliance family feud heat ongoing public...,174,business
4,1020,boro suffer morrison injury blow middlesbrough...,boro suffer morrison injury blow middlesbrough...,boro suffer morrison injury blow middlesbrough...,boro suffer morrison injury blow middlesbrough...,boro suffer morrison injury blow middlesbrough...,"[boro, suffer, morrison, injury, blow, middles...","[boro, suffer, morrison, injury, blow, middles...",boro suffer morrison injury blow middlesbrough...,71,sport


In [ ]:
# Edit dataframe to submit prediction results to Kaggle
submit_df = test.copy()
submit_df.rename(columns={'pred':'Category'}, inplace=True)
submit_df = submit_df[['ArticleId', 'Category']]

In [ ]:
submit_df.head()

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,sport
3,459,business
4,1020,sport


In [ ]:
submit_df.to_csv("/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Unsupervised learning/week4_peer_review/BBC News solution.csv", index=False)

# Supervised Learning case

In [ ]:
# I used tfidfvectorizer that trained previously, but newly trained
tfidvec_df = best_tfidvec.transform(train['lem_sent'])

In [ ]:
train.shape

(1484, 19)

In [ ]:
from sklearn.ensemble import AdaBoostClassifier

In [ ]:
train.head()

,Unnamed: 0,ArticleId,Text,Category,n_word_token,word_tokens,majority,majority_n,no_punct,no_punct_num,no_stopwords,clean_text,tokenized,lemmatized,lem_sent,n_words,target,best
0,0,1833,worldcom ex-boss launches defence lawyers defe...,business,18,"['last', 'week', 'mci', 'agreed', 'to', 'a', '...",a,2,worldcom exboss launches defence lawyers defen...,worldcom exboss launches defence lawyers defen...,worldcom exboss launches defence lawyers defen...,worldcom exboss launches defence lawyers defen...,"['worldcom', 'exboss', 'launches', 'defence', ...","['worldcom', 'exboss', 'launch', 'defence', 'l...",worldcom exboss launch defence lawyer defendin...,185,2,2
1,1,154,german business confidence slides german busin...,business,28,"['eurozone', 'interest', 'rates', 'are', 'at',...",interest,2,german business confidence slides german busin...,german business confidence slides german busin...,german business confidence slides german busin...,german business confidence slides german busin...,"['german', 'business', 'confidence', 'slides',...","['german', 'business', 'confidence', 'slide', ...",german business confidence slide german busine...,194,2,2
2,2,1101,bbc poll indicates economic gloom citizens in ...,business,14,"['in', 'eight', 'of', 'the', 'countries', 'the...",the,2,bbc poll indicates economic gloom citizens in ...,bbc poll indicates economic gloom citizens in ...,bbc poll indicates economic gloom citizens maj...,bbc poll indicates economic gloom citizens maj...,"['bbc', 'poll', 'indicates', 'economic', 'gloo...","['bbc', 'poll', 'indicates', 'economic', 'gloo...",bbc poll indicates economic gloom citizen majo...,267,2,2
3,3,1976,lifestyle governs mobile choice faster bett...,tech,23,"['by', 'contrast', 'he', 'said', 'camera', 'ph...",were,2,lifestyle governs mobile choice faster bett...,lifestyle governs mobile choice faster bett...,lifestyle governs mobile choice faster better ...,lifestyle governs mobile choice faster better ...,"['lifestyle', 'governs', 'mobile', 'choice', '...","['lifestyle', 'governs', 'mobile', 'choice', '...",lifestyle governs mobile choice faster better ...,344,4,4
4,4,917,enron bosses in $168m payout eighteen former e...,business,42,"['the', 'university', 'of', 'california', 'sai...",the,5,enron bosses in 168m payout eighteen former en...,enron bosses in m payout eighteen former enron...,enron bosses payout eighteen former enron dire...,enron bosses payout eighteen former enron dire...,"['enron', 'bosses', 'payout', 'eighteen', 'for...","['enron', 'boss', 'payout', 'eighteen', 'forme...",enron boss payout eighteen former enron direct...,198,2,2


In [ ]:
# convert category data into numerical one
train['Category_numeric'] = train['Category'].replace(train['Category'].unique(), np.arange(len(train['Category'].unique())))

In [ ]:
# Using Randomized search cross validation, find the best parameters for AdaBoost model
param_dist = {'learning_rate':np.arange(0.001, 1, 0.001),
                       'random_state':[42],
                       'n_estimators':np.arange(1,100,1)
                       }
rcl = AdaBoostClassifier()
rcl_random = RandomizedSearchCV(estimator=rcl, param_distributions=param_dist, n_iter=200, cv=3, verbose=2, random_state=42, n_jobs=-1)
rcl_random.fit(tfidvec_df, train['Category_numeric'])

Fitting 3 folds for each of 200 candidates, totalling 600 fits


RandomizedSearchCV(cv=3, estimator=AdaBoostClassifier(), n_iter=200, n_jobs=-1,
                   param_distributions={'learning_rate': array([0.001, 0.002, 0.003, 0.004, 0.005, 0.006, 0.007, 0.008, 0.009,
       0.01 , 0.011, 0.012, 0.013, 0.014, 0.015, 0.016, 0.017, 0.018,
       0.019, 0.02 , 0.021, 0.022, 0.023, 0.024, 0.025, 0.026, 0.027,
       0.028, 0.029, 0.03 , 0.031, 0.032, 0.033, 0.034, 0.035, 0.036,
       0.037, 0.038, 0.039...
                                        'n_estimators': array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68,
       69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85,
       86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99]),
                                        'random_state': [

In [ ]:
rcl_random.best_estimator_

AdaBoostClassifier(learning_rate=0.07, n_estimators=80, random_state=42)

In [ ]:
rcl_random.best_score_

0.812031243610191

In [ ]:
# Assign AdaBoost classifier using the best parameters
best_rcl = AdaBoostClassifier(learning_rate=0.07,
                              n_estimators=80,
                              random_state=42
                              )

In [ ]:
# Fit the model with train dataset
best_rcl.fit(tfidvec_df, train['Category_numeric'])

AdaBoostClassifier(learning_rate=0.07, n_estimators=80, random_state=42)

In [ ]:
# Predict label using the best model
pred = best_rcl.predict(tfidvec_df)

In [ ]:
accuracy_score(train['Category_numeric'], pred)

0.8807277628032345

In [ ]:
f1_score(train['Category_numeric'], pred, average='macro')

0.8769373715995649

### Predict test set

In [ ]:
tfidvec_test_df = best_tfidvec.transform(test['lem_sent'])

In [ ]:
pred_test = best_rcl.predict(tfidvec_test_df)

In [ ]:
pred_map = {i:c for c,i in zip(train['Category'].unique(), np.arange(len(train['Category'].unique())))}

In [ ]:
submit_df2 = submit_df.copy()

In [ ]:
submit_df2.head()

,ArticleId,Category
0,1018,sport
1,1319,tech
2,1138,sport
3,459,business
4,1020,sport


In [ ]:
submit_df2.shape

(735, 2)

In [ ]:
submit_df2['Category'] = list(pred_test)
submit_df2['Category'] = submit_df2['Category'].map(pred_map)

In [ ]:
submit_df2.to_csv("/content/drive/MyDrive/ColoradoBoulder/Machine Learning/Unsupervised learning/week4_peer_review/BBC News solution_ada.csv", index=False)

# Performance Summary Table
## NMF model
|Max_df|Min_df|Norm|Train Accuracy|Test Accuracy|
|------|------|----|--------------|-------------|
|0.05|0.00|L1|0.65|-
|0.10|0.00|L1|0.73|-
|0.15|1.00|L1|0.94|0.94|

## Adaboost model
|Learning_rate|n_estimator|Train Accuracy|Test Accuracy|
|-------------|-----------|--------------|-------------|
|0.082        |96         |0.88          |0.88         |